In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Load the data

In [ ]:
df = pd.read_csv('../input/covid19-case-surveillance-public-use-dataset/COVID-19_Case_Surveillance_Public_Use_Data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

Dropping the date columns and then the null values from dataset

In [ ]:
df = df.drop(['pos_spec_dt','onset_dt'],axis =1)

In [ ]:
df = df.dropna()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
colums = ['current_status', 'sex', 'age_group', 'Race and ethnicity (combined)', 'hosp_yn','icu_yn', 'death_yn', 'medcond_yn']
for col in colums:
    print(col)
    print(df[colums].value_counts())
    print("______________________")

In [ ]:
df.describe().T

In [ ]:
#Unique values in data
df.nunique()

## Some Interesting Insights from Visualization

In [ ]:
plt.figure(figsize=(30,10))
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9,
                      wspace=0.5, hspace=0.2)
plt.subplot(141)
plt.title('medcond_yn ',fontsize = 20)
df['medcond_yn'].value_counts().plot.pie(autopct="%1.1f%%")

In [ ]:
plt.figure(figsize=(30,10))
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9,
                      wspace=0.5, hspace=0.2)
plt.subplot(141)
plt.title('death_yn',fontsize = 20)
df['death_yn'].value_counts().plot.pie(autopct="%1.1f%%")

In [ ]:
plt.figure(figsize=(30,10))
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9,
                      wspace=0.5, hspace=0.2)
plt.subplot(141)
plt.title('hosp_yn',fontsize = 20)
df['hosp_yn'].value_counts().plot.pie(autopct="%1.1f%%")

In [ ]:
plt.figure(figsize=(30,10))
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9,
                      wspace=0.5, hspace=0.2)
plt.subplot(141)
plt.title('icu_yn',fontsize = 20)
df['icu_yn'].value_counts().plot.pie(autopct="%1.1f%%")

From the above pie charts we can see there are lot of missing data to conclude.

In [ ]:
plt.figure(figsize=(30,10))
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9,wspace=0.5, hspace=0.2)
plt.subplot(141)
plt.title('Race and ethnicity (combined)',fontsize = 20)
df['Race and ethnicity (combined)'].value_counts().plot.pie(autopct="%1.1f%%")

Pie chart on the basis of Ethnicity/Race

In [ ]:
plt.figure(figsize=(30,10))
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9,
                      wspace=0.5, hspace=0.2)
plt.subplot(141)
plt.title('current_status',fontsize = 20)
df['current_status'].value_counts().plot.pie(autopct="%1.1f%%")

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(y="current_status",hue ='sex',data=df)

Medical condition with respect to gender, while we have lot of missing data

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(y="medcond_yn",hue ='sex',data=df)

Death ratio of male and female are same, while missing rate is highest

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(y="death_yn",hue ='sex',data=df)

Age group from 20-29 has the highest count

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(y="age_group",data=df)

Age Group with respect to their gender.

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(y="age_group",hue ='sex',data=df)

In [ ]:
plt.figure(figsize=(10,8))
sns.catplot(x="age_group", hue="sex", col="current_status", data=df, kind="count", height=5, aspect=.7);

In [ ]:
print("Start Date:", df['cdc_report_dt'].min())
print("End Date:", df['cdc_report_dt'].max())

In [ ]:
df['age_group'].value_counts()

## Data Preparation For Model

In [ ]:
data = df.copy()

In [ ]:
data['sex'].value_counts()

### Converting categorical feature to numeric

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

Converting a categorical feature
Now we can convert features which contain strings to numerical values. This is required by most model algorithms. Doing so will also help us in achieving the feature completing goal.

Let us start by converting Sex feature to a new feature called Gender where female=0 and male=1 and so on.

In [ ]:
# mapp = {'Female':1,'Male':2,'Unknown':3,'Missing':4,'Other':5}
# data['sex'] = data['sex'].apply(lambda x:mapp[x])

data['sex'] = data['sex'].map({'Female':0,'Male':1,'Unknown':2,'Missing':3,'Other':4})
print(data.head())

In [ ]:
data["current_status"] = lb_make.fit_transform(data["current_status"])
data["hosp_yn"] = lb_make.fit_transform(data["hosp_yn"])
data["icu_yn"] = lb_make.fit_transform(data["icu_yn"])
data["death_yn"] = lb_make.fit_transform(data["death_yn"])
data["medcond_yn"] = lb_make.fit_transform(data["medcond_yn"])

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
X =  data[['current_status','hosp_yn','icu_yn','medcond_yn','sex']]
y = data['death_yn']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Model Building with Logistic Regression and Decision Tree:

Logistic Regression is a useful model to run early in the workflow. Logistic regression measures the relationship between the categorical dependent variable (feature) and one or more independent variables (features) by estimating probabilities using a logistic function, which is the cumulative logistic distribution

In [ ]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, y_train) * 100, 2)
acc_log

This model uses a decision tree as a predictive model which maps features (tree branches) to conclusions about the target value (tree leaves). Tree models where the target variable can take a finite set of values are called classification trees; in these tree structures, leaves represent class labels and branches represent conjunctions of features that lead to those class labels. Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees.

The model confidence score is the highest among models evaluated so far.

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, y_train) * 100, 2)
acc_decision_tree

#### Model evaluation
We can now rank our evaluation of all the models to choose the best one for our problem. While both Decision Tree and Logisitic Regression score the different, we choose to use Decision Tree, habit of overfitting to their training set.

In [ ]:
models = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree'],
    'Score': [ acc_log, acc_decision_tree]})
sorted_model=models.sort_values(by='Score', ascending=False)
sorted_model

In [ ]:
plt.figure(figsize=(20,10))
fig = plt.bar(sorted_model['Model'], sorted_model['Score'],color='aqua')
plt.grid()
plt.show()

## Can Try with below Models as well.

In [ ]:
# # Logistic Regression

# logreg = LogisticRegression()
# logreg.fit(X_train, Y_train)
# Y_pred = logreg.predict(X_test)
# acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
# acc_log

# # Support Vector Machines

# svc = SVC()
# svc.fit(X_train, Y_train)
# Y_pred = svc.predict(X_test)
# acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
# acc_svc


# knn = KNeighborsClassifier(n_neighbors = 3)
# knn.fit(X_train, Y_train)
# Y_pred = knn.predict(X_test)
# acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
# acc_knn

# # Gaussian Naive Bayes

# gaussian = GaussianNB()
# gaussian.fit(X_train, Y_train)
# Y_pred = gaussian.predict(X_test)
# acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
# acc_gaussian


# # Perceptron

# perceptron = Perceptron()
# perceptron.fit(X_train, Y_train)
# Y_pred = perceptron.predict(X_test)
# acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
# acc_perceptron



# # Linear SVC

# linear_svc = LinearSVC()
# linear_svc.fit(X_train, Y_train)
# Y_pred = linear_svc.predict(X_test)
# acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
# acc_linear_svc


# # Stochastic Gradient Descent

# sgd = SGDClassifier()
# sgd.fit(X_train, Y_train)
# Y_pred = sgd.predict(X_test)
# acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
# acc_sgd


# # Decision Tree

# decision_tree = DecisionTreeClassifier()
# decision_tree.fit(X_train, Y_train)
# Y_pred = decision_tree.predict(X_test)
# acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
# acc_decision_tree


# # Random Forest

# random_forest = RandomForestClassifier(n_estimators=100)
# random_forest.fit(X_train, Y_train)
# Y_pred = random_forest.predict(X_test)
# random_forest.score(X_train, Y_train)
# acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
# acc_random_forest

# models = pd.DataFrame({
#     'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
#               'Random Forest', 'Naive Bayes', 'Perceptron', 
#               'Stochastic Gradient Decent', 'Linear SVC', 
#               'Decision Tree'],
#     'Score': [acc_svc, acc_knn, acc_log, 
#               acc_random_forest, acc_gaussian, acc_perceptron, 
#               acc_sgd, acc_linear_svc, acc_decision_tree]})
# sorted_model=models.sort_values(by='Score', ascending=False)
# sorted_model


In [ ]:
# submission = pd.DataFrame({
#         "PatientId": test_df["patient"],
#         "death_yn": Y_pred
#     })
# submission.to_csv('submission2.csv', index=False)

### Please upvote, if it helps :)